
### **Interthread Communication Using Condition Object**

The `Condition` object in Python provides a more advanced mechanism for interthread communication compared to the `Event` object. It represents a state change in the application (e.g., producing or consuming an item). Threads can wait for a specific condition and get notified once that condition occurs.

- A `Condition` object allows one or more threads to wait until notified by another thread.
- The `Condition` object is always associated with a lock (usually a `ReentrantLock`).

### **Features**
1. The `Condition` object has methods like `acquire()` and `release()` that internally call the corresponding methods of the associated lock.
2. It provides additional methods like `wait()`, `notify()`, and `notifyAll()` for interthread communication.

#### **Creating a Condition Object**


In [ ]:
import threading

condition = threading.Condition()


### **Methods of `Condition`**

| Method            | Description                                                                 |
|--------------------|-----------------------------------------------------------------------------|
| `acquire()`        | Acquires the `Condition` object before performing actions (like producing/consuming). It acquires the internal lock. |
| `release()`        | Releases the `Condition` object after completing actions. It releases the internal lock. |
| `wait()`/`wait(time)` | Waits until receiving a notification or until the specified time expires. |
| `notify()`         | Notifies one waiting thread.                                               |
| `notifyAll()`      | Notifies all waiting threads.                                              |



### **Case Study: Producer-Consumer Problem**

### **Producer Thread**
The producer thread:
1. Acquires the `Condition`.
2. Produces an item.
3. Adds the item to the resource.
4. Notifies the consumer thread(s) that a new item is available.
5. Releases the `Condition`.

```python
# Producer Thread
condition.acquire()
# Produce item
...  # Generate the item
# Add item to the resource
condition.notify()  # Signal that a new item is available (use notifyAll() if multiple consumers)
condition.release()
```

### **Consumer Thread**
The consumer thread:
1. Acquires the `Condition`.
2. Waits for a notification.
3. Consumes the item once notified.
4. Releases the `Condition`.

```python
# Consumer Thread
condition.acquire()
condition.wait()  # Wait until the producer notifies
# Consume item
...  # Process the consumed item
condition.release()
```


### **Producer-Consumer Flow**
1. A producer thread generates an item and adds it to a shared resource (e.g., a queue or buffer).
2. It uses `notify()` to inform the consumer thread(s) that an item is ready.
3. The consumer thread(s), waiting on the `Condition`, are notified and proceed to consume the item.

By using the `Condition` object, producers and consumers can synchronize their actions efficiently.



In [1]:
from threading import Condition, Thread

def consume(c):
    c.acquire()  # Acquires the condition lock.
    print("Consumer waiting for updation")
    c.wait()  # Waits for a notification from the producer.
    print("Consumer got notification & consuming the item")
    c.release()  # Releases the condition lock.

def produce(c):
    c.acquire()  # Acquires the condition lock.
    print("Producer Producing Items")
    print("Producer giving Notification")
    c.notify()  # Sends a notification to the waiting consumer thread.
    c.release()  # Releases the condition lock.

c = Condition()  # Creates a Condition object for synchronization.

# Creates two threads: one for consuming and one for producing.
t1 = Thread(target=consume, args=(c,))
t2 = Thread(target=produce, args=(c,))

t1.start()  # Starts the consumer thread.
t2.start()  # Starts the producer thread.


Consumer waiting for updation
Producer Producing Items
Producer giving Notification
Consumer got notification & consuming the item


In [ ]:
from threading import Condition, Thread
import time
import random

# Shared resource
items = []

# Producer function
def produce(c):
    while True:
        c.acquire()  # Acquire the condition lock
        item = random.randint(1, 100)  # Generate a random item
        print(f"Producer Producing Item: {item}")
        items.append(item)  # Add the item to the shared list
        print("Producer giving Notification")
        c.notify()  # Notify the consumer that an item is available
        c.release()  # Release the condition lock
        time.sleep(5)  # Sleep for 5 seconds before producing the next item

# Consumer function
def consume(c):
    while True:
        c.acquire()  # Acquire the condition lock
        print("Consumer waiting for updation")
        c.wait()  # Wait for the producer's notification
        item = items.pop()  # Consume (remove) the last item from the list
        print(f"Consumer consumed the item: {item}")
        c.release()  # Release the condition lock
        time.sleep(5)  # Sleep for 5 seconds before consuming the next item

# Main execution
if __name__ == "__main__":
    c = Condition()  # Create a Condition object

    # Create threads for producer and consumer
    producer_thread = Thread(target=produce, args=(c,))
    consumer_thread = Thread(target=consume, args=(c,))

    # Start the threads
    producer_thread.start()
    consumer_thread.start()

    # Wait for threads to complete (optional for infinite loops)
    producer_thread.join()
    consumer_thread.join()


- The **consumer thread** is responsible for **calling `wait()`** on the `Condition` object because it needs to pause until the producer provides an update.
- The **producer thread** is responsible for **calling `notify()` or `notifyAll()`** on the `Condition` object because it updates the shared resource and signals the consumer to proceed.

This separation of responsibilities ensures smooth coordination between the producer and consumer threads.

### **Interthread Communication by Using Queue**

Queues are one of the most advanced mechanisms for interthread communication and for sharing data between threads. Here's an overview:


### **Advantages of Queues**
- **Built-in Synchronization**:  
  Queue internally uses a `Condition` object, which in turn uses a `Lock`. This eliminates the need for explicit synchronization when using queues.
  
- **Ease of Use**:  
  By using the queue module, synchronization complexities are automatically handled.



### **Importing the Queue Module**
To use queues, you must first import the `queue` module:  
```python
import queue
```


### **Creating a Queue Object**
To create a queue object:  
```python
q = queue.Queue()
```


### **Important Methods of Queue**
1. **`put()`**:  
   Used by the **Producer Thread** to insert data into the queue.  
   - Automatically acquires the lock before inserting data.  
   - Releases the lock after insertion.  
   - If the queue is full, the producer thread will wait internally using `wait()`.

2. **`get()`**:  
   Used by the **Consumer Thread** to remove and retrieve data from the queue.  
   - Automatically acquires the lock before removing data.  
   - Releases the lock after removal.  
   - If the queue is empty, the consumer thread will wait internally using `wait()`.  
   - Once the queue is updated, the consumer thread is notified automatically.


### **Producer Thread Logic**
The producer thread uses the `put()` method:  
1. Acquires the lock automatically.  
2. Checks if the queue is full.  
3. If not full, adds data to the queue.  
4. Releases the lock automatically.


### **Consumer Thread Logic**
The consumer thread uses the `get()` method:  
1. Acquires the lock automatically.  
2. Checks if the queue is empty.  
3. If not empty, retrieves and removes data from the queue.  
4. Releases the lock automatically.  
5. If the queue is empty, the thread waits until notified.


The `queue` module takes care of **locking** for us, which is a significant advantage, as it simplifies interthread communication and synchronization.

### **Python Supports**
1. FIFO Queue
2. LIFO Queue
3. Priority Queue

1. FIFO Queue:
In which order we put items into the queue, in the same order the items will come
out (FIFO)

In [2]:
import queue
q=queue.Queue()
q.put(10)
q.put(5)
q.put(15)
q.put(20)

while not q.empty():
    print(q.get(), end=' ')

10 5 15 20 

#### **LIFO Queue**

In [3]:
import queue
q=queue.LifoQueue()
q.put(10)
q.put(5)
q.put(15)
q.put(20)

while not q.empty():
    print(q.get(), end=' ')

20 15 5 10 

#### **Priority Queue**

In [5]:
import queue
q = queue.PriorityQueue()
# Add items to the queue as (priority, value)
q.put((2, 10))
q.put((1, 5))
q.put((3, 15))
q.put((4, 20))

while not q.empty():
    print(q.get(), end=' ')


(1, 5) (2, 10) (3, 15) (4, 20) 

In [9]:
import queue

# Create a PriorityQueue
q = queue.PriorityQueue()

# Add items to the queue with priorities
q.put((3, "Task C"))
q.put((1, "Task A"))
q.put((4, "Task D"))
q.put((2, "Task B"))

# Retrieve and print items in priority order
while not q.empty():
    _, task = q.get()
    print(f"{task}")


Task A
Task B
Task C
Task D
